# BertViz for clinical negation

In [ ]:
from bertviz import model_view
from transformers import (
    AutoConfig,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
)

from cnlpt.CnlpModelForClassification import CnlpModelForClassification

In [ ]:
model_name = "tmills/cnlpt-negation-roberta-sharpseed"
labels = ["-1", "1"]
max_length = 128
args = [
    "--output_dir",
    "bv_negation_out/",
    "--per_device_eval_batch_size",
    "128",
    "--do_predict",
    "--seed",
    "42",
    "--report_to",
    "none",
]
parser = HfArgumentParser((TrainingArguments,))
(training_args,) = parser.parse_args_into_dataclasses(args=args)

In [ ]:
import torch
import torch.backends.mps

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, config=config)
model = CnlpModelForClassification.from_pretrained(model_name, config=config)
model.to(device)
trainer = Trainer(model=model, args=training_args, compute_metrics=None)

In [ ]:
model_type = "roberta"

sentence_a = "The patient complains of dizziness and nausea but has no <e>pain</e> and denies anorexia."
sentence_b = ""

inputs = tokenizer.encode(sentence_a, return_tensors="pt")

In [ ]:
print(inputs)
outputs = model(inputs.to(device), output_attentions=True)

In [ ]:
print(len(outputs))
attention = outputs[-1]  # Output includes attention weights when output_attentions=True

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(inputs[0])

In [ ]:
# print(attention.shape)
print(tokens)
model_view(attention, tokens)

In [ ]:
from bertviz import head_view

head_view(attention, tokens)

In [ ]:
from bertviz import model_view

model_view(attention, tokens)